<a href="https://colab.research.google.com/github/dahiyajoy/SPEECH_DIARIZATION/blob/Master/Speech_Diarization_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Speech Diarizartion**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/Audio Trimmed.mp3'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
num_speakers = 2

language = 'English'

model_size = 'large'

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [5]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch


In [8]:
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

#Try to use the GPU, if available and driver is correctly installed. Otherwise, default to CPU.
try:
  embedding_model = PretrainedSpeakerEmbedding(
      "speechbrain/spkrec-ecapa-voxceleb",
      device=torch.device("cuda"))
except RuntimeError:
  print("No GPU found or driver issue. Defaulting to CPU.")
  embedding_model = PretrainedSpeakerEmbedding(
      "speechbrain/spkrec-ecapa-voxceleb",
      device=torch.device("cpu"))


from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [9]:
if file_path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', file_path, 'audio.wav', '-y'])
  file_path = 'audio.wav'

In [10]:
model = whisper.load_model(model_size)

In [11]:
result = model.transcribe(file_path)
segments = result["segments"]

In [12]:
with contextlib.closing(wave.open(file_path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [20]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(file_path, clip)
  return embedding_model(waveform[None])

In [22]:
import numpy as np
import torch

audio = Audio()

def segment_embedding(segment):
    start = segment["start"]
    # Whisper overshoots the end timestamp in the last segment
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(file_path, clip)

    # Ensure waveform is mono
    if waveform.ndim > 1 and waveform.shape[0] > 1:
        # Convert to mono by averaging channels using PyTorch
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    return embedding_model(waveform[None])

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)


In [23]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [24]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [25]:
# Load the contents of the .txt file
with open('transcript.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

print(file_content)


SPEAKER 1 0:00:00
Hi everybody, it's Curtis and I'm here with Amy from Real English Conversations and we're going to talk about driving in Canada and North America. And it all starts with getting your learner's license, right Amy? 
SPEAKER 2 0:00:17
Yeah, definitely. In order to be on the road behind the wheel of a vehicle, you've got to have a license. And the first thing that you do is you actually, at least in Canada, we do things in a very specific way. We have something called a graduated licensing program. I'm sure that the United States does it differently. But you pick up this book and it has all the rules of the road. 
SPEAKER 1 0:00:45
Yeah, and you have to study it because you have to write a written test or something like that. 
SPEAKER 2 0:00:52
Yeah, it's been a long time, right Curtis? 
SPEAKER 1 0:00:54
It's been a very long time for me, yeah. 
SPEAKER 2 0:01:00
So, yeah, you have to go in. There's a test that makes sure that obviously you've studied the book, you know

# **Speech Summarization**




In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
!pip install transformers

from transformers import pipeline

# Initialize the summarizer pipeline
summarizer = pipeline("summarization")

# Load the contents of the .txt file
with open('transcript.txt', 'r') as file:
    file_content = file.read()

# Remove occurrences of "SPEAKER 1", "SPEAKER 2", "SPEAKER 3"
content_cleaned = file_content.replace('SPEAKER 1', '').replace('SPEAKER 2', '').replace('SPEAKER 3', '')

# Generate the summary
summary = summarizer(content_cleaned, max_length=150, min_length=40, do_sample=False)

# Save the summary to a new file
with open('summary.txt', 'w', encoding='utf-8') as summary_file:
    summary_file.write(summary[0]['summary_text'])

# Print the summary
print("\nSummary of the Conversation:\n", summary[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Summary of the Conversation:
  In Canada, at least in Canada, we do things in a very specific way . We have something called a graduated licensing program . They give you a license which you're able to drive a vehicle under the supervision of someone who has a proper license .


# **Speech Translation to Arabic**


In [36]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=b14340b1a40d52cbae6798d2bd419c3882b85a8825642ecac6bfa6866fd6fb59
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


## **Summary Translation**

In [41]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Load the contents of the .txt file
with open('summary.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Translate the content to Arabic
translated = translator.translate(file_content, dest='ar')

# Save the Translated Transcripts to a new file
with open('summary_arabic.txt', 'w', encoding='utf-8') as output_file2:
    output_file2.write(translated.text)  # Use translated.text to get the string

# Print the original and translated content (optional)
print("Original Summary:\n", file_content)
print("\nTranslated Summary:\n", translated.text)


Original Summary:
  In Canada, at least in Canada, we do things in a very specific way . We have something called a graduated licensing program . They give you a license which you're able to drive a vehicle under the supervision of someone who has a proper license .

Translated Summary:
 في كندا ، على الأقل في كندا ، نفعل الأشياء بطريقة محددة للغاية.لدينا شيء يسمى برنامج الترخيص المتدرج.يعطونك ترخيصًا قادرًا على قيادة سيارة تحت إشراف شخص لديه ترخيص مناسب.


## **Transcript Translation**

In [42]:
from googletrans import Translator

# Initialize the Translator
translator = Translator()

# Load the contents of the .txt file
with open('transcript.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# Translate the content to Arabic
translated = translator.translate(file_content, dest='ar')

# Save the Translated Transcripts to a new file
with open('transcript_arabic.txt', 'w', encoding='utf-8') as output_file2:
    output_file2.write(translated.text)  # Use translated.text to get the string

# Print the original and translated content (optional)
print("\nTranslated Trnascripts:\n", translated.text)



Translated Trnascripts:
 المتحدث 1 0:00:00
مرحباً بالجميع ، إنه كورتيس وأنا هنا مع إيمي من محادثات اللغة الإنجليزية الحقيقية وسنتحدث عن القيادة في كندا وأمريكا الشمالية.ويبدأ كل شيء بالحصول على رخصة المتعلم ، أليس كذلك؟
المتحدث 2 0:00:17
نعم بالتأكيد.لكي تكون على الطريق خلف عجلة السيارة ، يجب أن يكون لديك ترخيص.وأول ما تفعله هو أنك في الواقع ، على الأقل في كندا ، نفعل الأشياء بطريقة محددة للغاية.لدينا شيء يسمى برنامج الترخيص المتدرج.أنا متأكد من أن الولايات المتحدة تفعل ذلك بشكل مختلف.لكنك تلتقط هذا الكتاب ولديه جميع قواعد الطريق.
المتحدث 1 0:00:45
نعم ، وعليك أن تدرسه لأنك يجب أن تكتب اختبارًا مكتوبًا أو شيء من هذا القبيل.
المتحدث 2 0:00:52
نعم ، لقد مر وقت طويل ، كورتيس صحيح؟
المتحدث 1 0:00:54
لقد مر وقت طويل جدًا بالنسبة لي ، نعم.
المتحدث 2 0:01:00
لذا ، نعم ، عليك أن تدخل. هناك اختبار يتأكد من الواضح أنك درست الكتاب ، أنت تعرف عن قواعد جعل منعطفًا يمينيًا ، على سبيل المثال ، عندما تحتاج إلى العائد ، عندما تحتاجللتوقف ، كل هذا النوع من الأشياء.
المتحدث 1 0:01:19
موازنة موازنة؟
المت